In [14]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
from bs4 import BeautifulSoup


- define the input: a string containing the full paper
- choose a standard commonly used NLP library (https://www.stxnext.com/blog/top-python-nlp-libraries/) nltk // textblob
- add as many features as possible to each word of the dictionary
- look for the features that correlates the most with the 'count' <---- put any machine learning here 
- use those features to guess the difficulty on the article
- for the top N most difficult words you match the meaning with the API

In [7]:
dict_df = pd.read_csv("unigram_freq.csv")

In [31]:
dict_df[dict_df["word"] == "cnn"]

,word,count,difficulty_level,definition
6578,cnn,9199332,NaN,NaN


In [29]:
dict_df.head(5)

,word,count,difficulty_level,definition
0,the,23135851162,A1,used before nouns to refer to particular thing...
1,of,13151942776,A1,"used to show possession, belonging, or origin:"
2,and,12997637966,A1,"used to join two words, phrases, parts of sent..."
3,to,12136980858,A1,used before a verb to show that it is in the i...
4,a,9081174698,A1,the first letter of the English alphabet


In [13]:
dict_df["word"].sample(5)

139656           asmp
30831      metallurgy
53190            lodz
130919    comradeship
56470        foretold
Name: word, dtype: object

In [28]:
import random

In [23]:
def scrape_word(word):
    user_agents = [
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
        'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.2 Safari/605.1.15',
        'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36',
        # Add more User-Agent strings here
    ]
    headers = {
        'User-Agent': random.choice(user_agents)
    }
    url = f"https://dictionary.cambridge.org/dictionary/english/{word}"
    response = requests.get(url, headers=headers)
    if response.status_code == 200:

        soup = BeautifulSoup(response.content, 'html.parser')
        difficulty_level = "Difficulty level not found"
        difficulty_span = soup.select_one(
            '.epp-xref.dxref.A1, .epp-xref.dxref.A2, .epp-xref.dxref.B1, .epp-xref.dxref.B2, .epp-xref.dxref.C1, .epp-xref.dxref.C2')
        if difficulty_span:
            difficulty_level = difficulty_span.get_text(strip=True)
        else:
            # Attempt to find difficulty level using the .gb.dgc class as a fallback
            difficulty_span_fallback = soup.select_one('.gc.dgc')
            if difficulty_span_fallback:
                difficulty_level = difficulty_span_fallback.get_text(strip=True)
            else:
                difficulty_level = "B"

        definition_div = soup.find('div', class_='def ddef_d db')
        definition = definition_div.text if definition_div else "Definition not found"

        return {"word": word, "difficulty_level": difficulty_level, "definition": definition}
    else:
        return {"word": word, "difficulty_level": "Error", "definition": ""}


In [26]:
for index, row in dict_df.iterrows():
    word_data = scrape_word(row["word"])
    dict_df.loc[index, "difficulty_level"] = word_data["difficulty_level"]
    dict_df.loc[index, "definition"] = word_data["definition"]

ConnectionError: HTTPSConnectionPool(host='dictionary.cambridge.org', port=443): Max retries exceeded with url: /dictionary/english/centre (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000002808ADBEA10>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))

In [27]:
len(dict_df[dict_df["difficulty_level"].notna()])

823